In [1]:
import pandas as pd
import os
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
# Veriyi yükleme
df = pd.read_excel(r"C:\Users\EXCALIBUR\Desktop\Çalışmalar\Chinook_DB\Chinook\data.xlsx")


In [3]:
df.head()

,CustomerID,CustomerName,Country,AlbumName,ArtistName,SongName,GenreName
0,1,Luís Gonçalves,Brazil,Acústico MTV,Os Paralamas Do Sucesso,Vamo Batê Lata,Latin
1,1,Luís Gonçalves,Brazil,Acústico MTV [Live],Cidade Negra,A Cor Do Sol,Reggae
2,1,Luís Gonçalves,Brazil,Acústico MTV [Live],Cidade Negra,Podes Crer,Reggae
3,1,Luís Gonçalves,Brazil,Afrociberdelia,Chico Science & Nação Zumbi,Interlude Zumbi,Latin
4,1,Luís Gonçalves,Brazil,Appetite for Destruction,Guns N' Roses,Rocket Queen,Rock


In [4]:
# TF-IDF Uygulama
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df['SongName'] + " " + df['GenreName'] + " " + df['ArtistName'])


# Şarkılar arasında benzerlik
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [5]:
print(cosine_sim)

[[1.         0.         0.         ... 0.         0.         0.        ]
 [0.         1.         0.45739384 ... 0.         0.         0.        ]
 [0.         0.45739384 1.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 1.         0.         0.        ]
 [0.         0.         0.         ... 0.         1.         0.15087313]
 [0.         0.         0.         ... 0.         0.15087313 1.        ]]


In [6]:
import pandas as pd

def get_recommendations_with_previous_songs(customer_name, genre):
    # Kullanıcının önceki dinlediği şarkıları çek
    previous_songs = df[df['CustomerName'] == customer_name]
    
    # Kullanıcının daha önce dinlediği şarkıların türünü belirleyelim
    previous_songs_list = previous_songs[previous_songs['GenreName'] == genre]

    # Seçilen türdeki öneri şarkıları alalım
    genre_songs = df[df['GenreName'] == genre]
    
    # Kullanıcıya önerilen şarkıları almak için filtrele
    recommendations = genre_songs[genre_songs['CustomerName'] != customer_name]
    
    # En fazla 5 öneri alalım
    recommendations = recommendations[['SongName', 'AlbumName', 'ArtistName', 'GenreName']].head(5)

    # Pandas tablosu şeklinde görsel çıktı
    return previous_songs_list[['SongName', 'AlbumName', 'ArtistName', 'GenreName']], recommendations

In [23]:
# Kullanıcıdan alınan tür bilgisiyle öneri yapılması
customer_name = "Bjørn Hansen"
genre = "Rock"

In [24]:
previous_songs, recommendations = get_recommendations_with_previous_songs(customer_name, genre)


In [25]:
# Sonuçları güzelce yazdıralım
print(f"**{customer_name}**'ın Daha Önce Dinlediği {genre} Şarkıları:\n")
print(previous_songs.to_string(index=False))

print(f"\n**Önerilen {genre} Şarkıları:**\n")
print(recommendations.to_string(index=False))


**Bjørn Hansen**'ın Daha Önce Dinlediği Rock Şarkıları:

                               SongName                             AlbumName                   ArtistName GenreName
                           Born To Move                     Chronicle, Vol. 2 Creedence Clearwater Revival      Rock
                                  Plush                                  Core          Stone Temple Pilots      Rock
                     Breaking The Rules For Those About To Rock We Salute You                        AC/DC      Rock
                             Evil Walks For Those About To Rock We Salute You                        AC/DC      Rock
                       Inject The Venom For Those About To Rock We Salute You                        AC/DC      Rock
                  Put The Finger On You For Those About To Rock We Salute You                        AC/DC      Rock
                    How Many More Times                        Led Zeppelin I                 Led Zeppelin      Rock
       

In [26]:
import pickle

# Veriyi pickle ile kaydetmek
with open('song_recommendations.pkl', 'wb') as f:
    pickle.dump({'previous_songs': previous_songs, 'recommendations': recommendations}, f)


In [27]:
# Pickle dosyasını açma ve içeriğini kontrol etme
with open('song_recommendations.pkl', 'rb') as file:
    recommendations = pickle.load(file)

In [28]:
# Kullanıcıdan isim ve tür bilgisi alma
user_name = input("Adınızı girin: ")
genre = input("Hangi müzik türünde şarkı önerisi istersiniz? ")

# Kullanıcıya göre önerileri alma
recommendations_for_user = recommendations.get(genre, [])

# Sonuçları yazdırma
if recommendations_for_user:
    print(f"\nMerhaba {user_name}, işte {genre} türünde 5 şarkı öneriniz:")
    for idx, song in enumerate(recommendations_for_user[:5], 1):
        print(f"{idx}. {song['SongName']} - {song['AlbumName']} by {song['ArtistName']}")
else:
    print(f"\nÜzgünüm, {genre} türüne ait şarkı önerileri bulunamadı.")



Üzgünüm, Rock türüne ait şarkı önerileri bulunamadı.
